In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Sep 21 12:31:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    52W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install sentencepiece
!pip install transformers datasets 
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, ClassLabel, load_metric
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [4]:
def evaluate(pipeline, dataset_test, labels, output_dict=False):
    y_hat_proba = pipeline.predict(dataset_test)
    y_hat = np.argmax(y_hat_proba[0], axis=1)
    return classification_report(labels, y_hat, output_dict=output_dict)

In [5]:
def compute_metrics(eval_pred):
    metric = load_metric("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='micro')

In [6]:
#Importar drive para guardar modelo o importar datos
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
PATH ="/content/gdrive/MyDrive/Colab Notebooks/TFM/dataSet/raw_data_verbos.csv"
try:
    dataSet = pd.read_csv(PATH,sep=";", encoding='latin1',error_bad_lines=False)
except:
    print("error")
numClass = len(dataSet['Dialecto'].unique())

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MODEL_NAME = "ixa-ehu/roberta-eus-mc4-base-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,
                                                               num_labels=numClass).to(device)


Some weights of the model checkpoint at ixa-ehu/roberta-eus-mc4-base-cased were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ixa-ehu/roberta-eus-mc4-base-cased and are newly initialized: ['classifier.out_proj.weight',

In [9]:
N_SAMPLES = 70
data = pd.DataFrame()
#data = dataSet.groupby('Dialecto', group_keys=False).apply(lambda x: x.sample(N_SAMPLES))
data[['text', 'label']] = dataSet[['Frase', 'Dialecto']]
#data = data.drop(columns=['Frase', 'Dialecto'], axis=1)
list_labels = data.label.unique().tolist()
features = ClassLabel(names=list_labels)
data_train, data_test = train_test_split(data, test_size=0.2, stratify=data.label, shuffle=True,random_state = 1)
label4,label0,label2,label1,label3 = data_train['label'].value_counts()
arrayLabels = np.array([label0, label1, label2, label3, label4])
maxLabels =np.amax(arrayLabels)
indiceLabel0 = round(maxLabels/label0,2)
indiceLabel1 = round(maxLabels/label1,2)
indiceLabel2 = round(maxLabels/label2,2)
indiceLabel3 = round(maxLabels/label3,2)
indiceLabel4 = round(maxLabels/label4,2)
print(indiceLabel0,indiceLabel1,indiceLabel2,indiceLabel3,indiceLabel4)

1.24 1.68 1.67 3.06 1.0


In [10]:

datasetTest = Dataset.from_pandas(data_test)

In [11]:
print(list_labels)

['Nafar-Lapurtarra', 'Erdialdekoa', 'Nafarra', 'Zuberotarra', 'Mendebaldekoa']


In [12]:
def tokenize(batch):
    tokens = tokenizer(batch['text'], padding=True, truncation=True, max_length=512)
    tokens['label'] = features.str2int(batch['label'])
    return tokens

In [13]:

datasetTest = datasetTest.map(tokenize, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [14]:
training_args = TrainingArguments(output_dir="/content/gdrive/MyDrive/Colab Notebooks/TFM/modelHF_frases",
                                      overwrite_output_dir=True,
                                      num_train_epochs=5,
                                      per_device_train_batch_size=1,
                                      evaluation_strategy="epoch",
                                      learning_rate=1e-6,
                                      save_strategy='no')

In [15]:
from torch import nn
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 5 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([np.float32(indiceLabel0), np.float32(indiceLabel1), np.float32(indiceLabel2),np.float32(indiceLabel3),np.float32(indiceLabel4)])).to(device)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [16]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,num_labels=numClass, return_dict=True)

In [17]:
def my_hp_space_optuna(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-7, 5e-6, log=True),
        "warmup_steps":  trial.suggest_float("warmup_steps", 0.0, 1.5, step=0.3),
        "weight_decay":  trial.suggest_float("weight_decay", 1e-6, 1e-1)
    }
def my_objective(metrics):
    return metrics["eval_f1"]

In [18]:
 from sklearn.model_selection import KFold



In [19]:
 kf = KFold(n_splits=5)
print(data_train)

                                                    text             label
11481          seguruago da beti dirua bankuan gordetzea           Nafarra
3206                                     isileko eizturi           Nafarra
3905                         gaueko abaro ihes egin zuen       Erdialdekoa
11917                                    dempora kultura     Mendebaldekoa
1721                                  salmon gustuko dut           Nafarra
...                                                  ...               ...
7806                                        neguan, negu  Nafar-Lapurtarra
6867                             elur teilatua estali du  Nafar-Lapurtarra
376    armiarma pizten dogunean, gelak askoz ere eder...           Nafarra
543    dendan gona oparitu didate, ehunak kakalarru z...       Erdialdekoa
8842      egun asteazkena da eta bulegora joan behar dut  Nafar-Lapurtarra

[9611 rows x 2 columns]


In [20]:
for train, test in kf.split(data_train):
  print("%s %s" % (train, test))
  train_df = data_train.iloc[train]

[1923 1924 1925 ... 9608 9609 9610] [   0    1    2 ... 1920 1921 1922]
[   0    1    2 ... 9608 9609 9610] [1923 1924 1925 ... 3842 3843 3844]
[   0    1    2 ... 9608 9609 9610] [3845 3846 3847 ... 5764 5765 5766]
[   0    1    2 ... 9608 9609 9610] [5767 5768 5769 ... 7686 7687 7688]
[   0    1    2 ... 7686 7687 7688] [7689 7690 7691 ... 9608 9609 9610]


In [21]:
train_df

,text,label
11481,seguruago da beti dirua bankuan gordetzea,Nafarra
3206,isileko eizturi,Nafarra
3905,gaueko abaro ihes egin zuen,Erdialdekoa
11917,dempora kultura,Mendebaldekoa
1721,salmon gustuko dut,Nafarra
...,...,...
2625,atzerrira informazioa pasatzen eun sator euen ...,Mendebaldekoa
1013,orratz baten bidez besoan üsüki dit,Zuberotarra
11640,"gainditu egin det eta, hala ere, kexuka dailll",Erdialdekoa
6089,posu,Mendebaldekoa


In [22]:
train = Dataset.from_pandas(train_df)

In [23]:
for train_index, test_index in kf.split(data_train):
  train_df = data_train.iloc[train_index]
  val_df = data_train.iloc[test_index]
  label4,label0,label2,label1,label3 = train_df['label'].value_counts()
  arrayLabels = np.array([label0, label1, label2, label3, label4])
  maxLabels =np.amax(arrayLabels)
  indiceLabel0 = round(maxLabels/label0,2)
  indiceLabel1 = round(maxLabels/label1,2)
  indiceLabel2 = round(maxLabels/label2,2)
  indiceLabel3 = round(maxLabels/label3,2)
  indiceLabel4 = round(maxLabels/label4,2)
  train = Dataset.from_pandas(train_df)
  test = Dataset.from_pandas(val_df)
  train = train.map(tokenize, batched=True)
  test = test.map(tokenize, batched=True)
  trainer = CustomTrainer(
          #model=model,
          args=training_args,
          train_dataset=train,
          eval_dataset=test,
          compute_metrics=compute_metrics,
          model_init = model_init
      )
  best_trial  = trainer.hyperparameter_search(
      direction="maximize", #maximizar porque en compute objetive tenemos una metrica que queremos mejorar, si fuese una funcion de perdida pondriamos minimizar
      n_trials=1,#numero de veces que chequea los datos de test
      hp_space=my_hp_space_optuna, 
      compute_objective=my_objective
  )
  print(best_trial)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--ixa-ehu--roberta-eus-mc4-base-cased/snapshots/6c04ab91eb1b8a457fbda51c9de22a395193cd2d/config.json
Model config RobertaConfig {
  "_name_or_path": "ixa-ehu/roberta-eus-mc4-base-cased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,

Epoch,Training Loss,Validation Loss,F1
1,1.454300,1.503135,0.433697
2,1.302500,1.353672,0.491940
3,1.269000,1.385921,0.512741
4,1.238800,1.414385,0.529901
5,1.340800,1.412159,0.525221


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1923
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1923
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.f

...

  0%|          | 0/2 [00:00<?, ?ba/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--ixa-ehu--roberta-eus-mc4-base-cased/snapshots/6c04ab91eb1b8a457fbda51c9de22a395193cd2d/config.json
Model config RobertaConfig {
  "_name_or_path": "ixa-ehu/roberta-eus-mc4-base-cased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,

Epoch,Training Loss,Validation Loss,F1
1,1.284800,1.364956,0.497919
2,1.271600,1.495935,0.536941
3,1.444800,1.656420,0.548907
4,1.366700,1.679685,0.561394
5,1.330900,1.754015,0.563996


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1922
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1922
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.f

BestRun(run_id='0', objective=0.5639958376690947, hyperparameters={'learning_rate': 1.0265099773663673e-06, 'warmup_steps': 0.3, 'weight_decay': 0.0425326749368974})


In [24]:
print(best_trial)

BestRun(run_id='0', objective=0.5639958376690947, hyperparameters={'learning_rate': 1.0265099773663673e-06, 'warmup_steps': 0.3, 'weight_decay': 0.0425326749368974})


In [25]:
#prediction = trainer.predict(datasetTest)
#report = evaluate(trainer, datasetTest, datasetTest['label'])
#print(report)